# Filter experiments by fraction within primary injection site

Use SDK function/cache

In [1]:
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

# !! update this path so that it points to your external hard drive !!
mcc = MouseConnectivityCache(manifest_file='D:/connectivity/manifest.json')

# get metadata for all the experiments
all_experiments = mcc.get_experiments(dataframe=True)
print "There are %d experiments in the cache" % len(all_experiments)

There are 2173 experiments in the cache


Get ontology information

In [2]:
ontology = mcc.get_ontology()

# Experiment Filtering

In [3]:
include_parent_structures = ['VIS','PTLp']
include_structures_id = ontology.get_descendant_ids(*ontology[include_parent_structures].id.values)

# some structure unionizes files on my disk are corrupt
exclude_experiments = [113887162]

experiment_mask = (
    all_experiments['structure-id'].isin(include_structures_id) & 
    ~all_experiments['id'].isin(exclude_experiments)
    )

filtered_experiments = all_experiments[experiment_mask]
print "There are %d experiments in my filtered experiment list" % len(filtered_experiments)

There are 179 experiments in my filtered experiment list


# Structure Filtering

In [4]:
# EXAMPLE: Use the SDK to fetch the coarse 12 structural brain division

from allensdk.api.queries.ontologies_api import OntologiesApi

summary_structures = OntologiesApi().get_structures(structure_set_names='Mouse Connectivity - Summary')
summary_structure_ids = [ s['id'] for s in summary_structures ]

# order the structures by ontology graph order
filtered_structures = ontology[summary_structure_ids].sort('graph_order')
print "There are %d structures in my filtered structure list" % len(filtered_structures)

There are 293 structures in my filtered structure list


# Use helper function to get injection matrix

In [5]:
import connectivity_helper as chelp

# normalized_projection_volume = fraction of injection is within the structure
inj_matrix = test = chelp.generate_matrix( mcc, filtered_experiments, filtered_structures, [3],
                            is_injection=True, parameter='normalized_projection_volume')

In [23]:
# compute the injectin fraction for the primary injection structure
import numpy as np

m = inj_matrix['matrix'].copy()
m[np.isnan(m)] = 0
primary_injection_fraction = np.amax(m,1)

lab = inj_matrix['columns']['label'].values
primary_injection_label = lab[np.argmax(m,1)]

# only keep experiments above threshold
fraction_threshold = 0.8
experiment_mask = primary_injection_fraction > fraction_threshold
keep_experiments = filtered_experiments[experiment_mask]

print "Keeping %d experiments" % len(keep_experiments)

Keeping 116 experiments


In [24]:
keep_experiments['structure-abbrev'].unique()

array([u'VISp', u'VISam', u'PTLp', u'VISal', u'VISl', u'VISpm', u'VISpl',
       u'VISp4'], dtype=object)

In [25]:
np.unique(primary_injection_label[experiment_mask])

array(['RSPagl', 'VISal', 'VISam', 'VISl', 'VISli', 'VISp', 'VISpl',
       'VISpm', 'VISpor', 'VISrl'], dtype=object)